In [1]:
import os
os.chdir("../")
!pwd

/home/aditya/network_security


In [2]:
from pathlib import Path
from pydantic import BaseModel

class ModelEvaluationConfig(BaseModel):
    ## from config
    root_dir: Path
    test_data_path: Path
    metric_file_name: str
    target_column: str
    mlflow_uri: str

In [3]:
from src.NetworkSecurity.constants import *
from src.NetworkSecurity.utils.common import read_yaml,create_directories


class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self)->ModelEvaluationConfig:

        config = self.config.model_evaluation
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        data_model_evaluation_config = ModelEvaluationConfig(

            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            mlflow_uri= config.mlflow_uri,
            metric_file_name = config.metric_file_name,
            target_column = schema.name
        )

        return data_model_evaluation_config

In [6]:
cm = ConfigurationManager()
model_evaluation_config = cm.get_model_evaluation_config()

[2025-03-31 15:56:56,893: INFO: common : Yaml File: config/config.yaml loaded successfully]
[2025-03-31 15:56:56,896: INFO: common : Yaml File: params.yaml loaded successfully]
[2025-03-31 15:56:56,899: INFO: common : Yaml File: schema.yaml loaded successfully]
[2025-03-31 15:56:56,900: INFO: common : created directory at: artifacts]
[2025-03-31 15:56:56,901: INFO: common : created directory at: artifacts/model_evaluation]


In [17]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import mlflow
from src.NetworkSecurity.utils.common import save_json

class ModelEvaluate:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self, actual, pred, pred_proba):
        accuracy = accuracy_score(actual, pred)
        precision = precision_score(actual, pred, average='weighted')
        recall = recall_score(actual, pred, average='weighted')
        f1 = f1_score(actual, pred, average='weighted')
        auc = roc_auc_score(actual, pred_proba, multi_class="ovr") if len(set(actual)) > 2 else roc_auc_score(actual, pred_proba)
        return accuracy, precision, recall, f1, auc

    def validate_model(self, model_uri, sample_input):
        """Validates the model before using it for evaluation."""
        try:
            mlflow.models.predict(
                model_uri=model_uri,
                input_data=sample_input,
                env_manager="uv",
            )
            print(" Model validation successful!")
        except Exception as e:
            print(f"Model validation failed: {e}")
            raise e
    
    def evaluate(self):
        test_data = pd.read_csv(self.config.test_data_path)
        x_test = test_data.drop([self.config.target_column], axis=1)
        y_test = test_data[self.config.target_column]

        mlflow.set_tracking_uri(self.config.mlflow_uri)
        mlflow.set_registry_uri(self.config.mlflow_uri)
        
        model_uri = "runs:/85e7a50505b049edbd32ebd27d71eeef/model"  # Update with latest run ID
        loaded_model = mlflow.pyfunc.load_model(model_uri)

        # Validate model
        sample_input = x_test.iloc[:1]  # Taking a single row for validation
        self.validate_model(model_uri, sample_input)

        # Predict
        y_pred = loaded_model.predict(pd.DataFrame(x_test))
        y_pred_proba = loaded_model.predict_proba(pd.DataFrame(x_test))[:, 1]  # If binary classification

        # Evaluate
        accuracy, precision, recall, f1, auc = self.eval_metrics(y_test, y_pred, y_pred_proba)

        scores = {
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1_score": f1,
            "auc": auc
        }

        save_json(self.config.metric_file_name,scores)
        print(f"Evaluation Metrics - Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}")


In [ ]:
me = ModelEvaluate(model_evaluation_config)
me.evaluate()